In [5]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [6]:
dfenduse = pd.read_excel("./data/HS2012-17-BEC5 -- 08 Nov 2018.xlsx", sheet_name="HS17BEC5", usecols = ["HS6", "BEC5EndUse"], dtype=str)

dfenduse["BEC5EndUse"].unique()

# Define a dictionary for replacements
replacement_dict = {
    "INT/CONS": "INT",
    'CONS/INT': "CONS",
    'CAP/INT': "CAP",
    "INT/CAP": "INT",
    'CAP/CONS': "CAP",
    'CONS/CAP': "CONS",
    np.nan: "OTHER",
}

dfenduse["BEC5EndUse"] = dfenduse["BEC5EndUse"].replace(replacement_dict)

dfenduse["BEC5EndUse"].unique()

dfenduse.to_parquet("./data/hs6-enduse.parquet", index=False)

In [ ]:
df = pd.read_parquet("data/TOTAldata-current.parquet")

# Create HS code hierarchy columns

df["HS6"] = df["I_COMMODITY"].str[0:6]

df["HS4"] = df["I_COMMODITY"].str[0:4]

df["HS2"] = df["I_COMMODITY"].str[0:2]

# HS8 level (must create column first)

df["HS8"] = df["I_COMMODITY"].str[0:8]

df["HS10"] = df["I_COMMODITY"]

# Convert time to datetime

df.time = pd.to_datetime(df.time, format="%Y-%m")

# Convert numeric columns to float

df["CON_VAL_MO"] = df["CON_VAL_MO"].astype(float)

df["CAL_DUT_MO"] = df["CAL_DUT_MO"].astype(float)

# Report annual trade shares by end use

enduse_map = pd.read_parquet("./data/hs6-enduse.parquet")[["HS6", "BEC5EndUse"]]

df_report = df.merge(enduse_map, on="HS6", how="left")

df_report["year"] = df_report["time"].dt.year


df_report["enduse_group"] = np.where(

    df_report["BEC5EndUse"].isin(["CAP", "INT", "CONS"]),

    df_report["BEC5EndUse"],

    "MISSING"

)


annual = (

    df_report.groupby(["year", "enduse_group"], as_index=False)["CON_VAL_MO"]

    .sum()

    .pivot(index="year", columns="enduse_group", values="CON_VAL_MO")

    .fillna(0)

)


for col in ["CAP", "INT", "CONS", "MISSING"]:

    if col not in annual.columns:

        annual[col] = 0.0


annual = annual[["CAP", "INT", "CONS", "MISSING"]]

annual_total = annual.sum(axis=1).replace(0, np.nan)

annual_shares = annual.div(annual_total, axis=0) * 100

annual_shares.columns = [

    "capital_share_pct",

    "intermediates_share_pct",

    "consumption_share_pct",

    "missing_trade_share_pct",

]

print("Annual shares of import value (%):")

print(annual_shares.round(2))


Annual shares of import value (%):
      capital_share_pct  intermediates_share_pct  consumption_share_pct  \
year                                                                      
2013              15.77                    45.60                  33.76   
2014              16.42                    44.99                  33.42   
2015              17.82                    42.27                  34.20   
2016              18.00                    41.50                  34.45   
2017              18.26                    43.29                  34.17   
2018              18.03                    44.49                  33.13   
2019              18.18                    43.72                  33.50   
2020              19.79                    43.38                  32.35   
2021              18.27                    44.69                  32.63   
2022              17.31                    43.56                  28.62   
2023              18.16                    42.48                 

In [ ]:
print('BEC5EndUse categories after mapping:')

print(dfenduse['BEC5EndUse'].value_counts())

print('\nTotal HS6 codes:', len(dfenduse))